# Practice Exercise Solutions

This notebook contains solutions to the practice exercises from the pandas introduction notebook. Use this as a reference after attempting the exercises yourself.

In [ ]:
# Import required libraries
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Set plot style
plt.style.use('seaborn')
%matplotlib inline

# Load the data
sales_df = pd.read_csv('../../datasets/sales/retail_sales_data.csv')
sales_df['Date'] = pd.to_datetime(sales_df['Date'])

## 1. Time Series Analysis Solutions

In [ ]:
# Calculate daily total sales
daily_sales = sales_df.groupby('Date')['Total_Sales'].sum().reset_index()

# Find busiest day of the week
sales_df['Day_of_Week'] = sales_df['Date'].dt.day_name()
dow_sales = sales_df.groupby('Day_of_Week')['Total_Sales'].agg(['sum', 'mean']).round(2)
print("Sales by Day of Week:")
print(dow_sales)

# Compare weekday vs weekend sales
sales_df['Is_Weekend'] = sales_df['Date'].dt.dayofweek.isin([5, 6])
weekend_comparison = sales_df.groupby('Is_Weekend')['Total_Sales'].agg(['sum', 'mean', 'count']).round(2)
weekend_comparison.index = ['Weekday', 'Weekend']
print("\nWeekday vs Weekend Sales:")
print(weekend_comparison)

## 2. Product Analysis Solutions

In [ ]:
# Calculate profit margins
sales_df['Cost'] = sales_df['Unit_Price'] * 0.6  # 40% margin
sales_df['Profit'] = sales_df['Total_Sales'] - (sales_df['Cost'] * sales_df['Quantity'])
sales_df['Profit_Margin'] = (sales_df['Profit'] / sales_df['Total_Sales'] * 100).round(2)

# Best-selling products
product_sales = sales_df.groupby('Product').agg({
    'Quantity': 'sum',
    'Total_Sales': 'sum',
    'Profit': 'sum',
    'Profit_Margin': 'mean'
}).round(2).sort_values('Quantity', ascending=False)

print("Product Performance:")
print(product_sales)

# Promotional effectiveness
promo_impact = sales_df.groupby(['Product', 'On_Promotion']).agg({
    'Quantity': 'mean',
    'Total_Sales': 'mean',
    'Profit_Margin': 'mean'
}).round(2)

print("\nPromotional Impact:")
print(promo_impact)

## 3. Regional Analysis Solutions

In [ ]:
# Regional sales rankings
regional_performance = sales_df.groupby('Region').agg({
    'Total_Sales': 'sum',
    'Quantity': 'sum',
    'Profit': 'sum'
}).round(2).sort_values('Total_Sales', ascending=False)

print("Regional Rankings:")
print(regional_performance)

# Best category by region
region_category = sales_df.groupby(['Region', 'Category'])['Total_Sales'].sum().unstack()
best_category = region_category.idxmax(axis=1)
print("\nBest Performing Category by Region:")
print(best_category)

# Regional seasonal patterns
sales_df['Month'] = sales_df['Date'].dt.month
seasonal_patterns = sales_df.groupby(['Region', 'Month'])['Total_Sales'].sum().unstack()

# Plot seasonal patterns
plt.figure(figsize=(15, 6))
seasonal_patterns.plot(marker='o')
plt.title('Regional Sales Patterns by Month')
plt.xlabel('Month')
plt.ylabel('Total Sales ($)')
plt.legend(title='Region')
plt.grid(True)
plt.show()

## 4. Advanced Dashboard Solutions

In [ ]:
# Create monthly performance dashboard
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(15, 12))

# 1. Monthly Sales Trend with Trend Line
monthly_trend = sales_df.groupby('Month')['Total_Sales'].sum()
x = range(1, 13)
z = np.polyfit(x, monthly_trend, 1)
p = np.poly1d(z)

ax1.plot(x, monthly_trend, 'bo-', label='Actual')
ax1.plot(x, p(x), 'r--', label='Trend')
ax1.set_title('Monthly Sales Trend')
ax1.set_xlabel('Month')
ax1.set_ylabel('Total Sales ($)')
ax1.legend()

# 2. Category Performance by Month
category_monthly = sales_df.pivot_table(
    values='Total_Sales',
    index='Month',
    columns='Category',
    aggfunc='sum'
)
category_monthly.plot(ax=ax2)
ax2.set_title('Category Performance')
ax2.set_xlabel('Month')
ax2.set_ylabel('Sales ($)')

# 3. Cumulative Sales
sales_df['Cumulative_Sales'] = sales_df.groupby('Category')['Total_Sales'].cumsum()
for category in sales_df['Category'].unique():
    data = sales_df[sales_df['Category'] == category]
    ax3.plot(data['Date'], data['Cumulative_Sales'], label=category)
ax3.set_title('Cumulative Sales by Category')
ax3.set_xlabel('Date')
ax3.set_ylabel('Cumulative Sales ($)')
ax3.legend()

# 4. Regional Market Share Over Time
regional_share = sales_df.pivot_table(
    values='Total_Sales',
    index='Month',
    columns='Region',
    aggfunc='sum'
).apply(lambda x: x/x.sum() * 100, axis=1)

regional_share.plot(kind='area', stacked=True, ax=ax4)
ax4.set_title('Regional Market Share')
ax4.set_xlabel('Month')
ax4.set_ylabel('Market Share (%)')

plt.tight_layout()
plt.show()